### Import the data and combine datasets across years

In [1]:
import pandas as pd
from datetime import date

yrs_list = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']
first_year = '2005'

def concat_mult_ref_tables(filename, yrs):
    """Return a dataframe that concatenates all 
    files across a list of years, with the year set as a key
    """
    # create a list to store the dfs
    df_list = []
    
    for yr in yrs: 
        temp_df = None   #clear out the df
        temp_df = pd.read_csv('./input/' + str(yr) + '/' + filename + '.csv')  #read in the file
        df_list.append(temp_df)
        
    final_df = pd.concat(df_list, keys=yrs)
    #final_df.rename_axis(['Season', 'Ix'])
    
    return final_df

# Create master dataframes with all the available years
conference_mstr = concat_mult_ref_tables('conference', yrs_list)
drive_mstr = concat_mult_ref_tables('drive', yrs_list)
game_statistics_mstr = concat_mult_ref_tables('game-statistics', yrs_list)
game_mstr = concat_mult_ref_tables('game', yrs_list)
kickoff_return_mstr = concat_mult_ref_tables('kickoff-return', yrs_list)
kickoff_mstr = concat_mult_ref_tables('kickoff', yrs_list)
pass_mstr = concat_mult_ref_tables('pass', yrs_list)
play_mstr = concat_mult_ref_tables('play', yrs_list)
player_game_statistics_mstr = concat_mult_ref_tables('player-game-statistics', yrs_list)
player_mstr = concat_mult_ref_tables('player', yrs_list)
punt_return_mstr = concat_mult_ref_tables('punt-return', yrs_list)
punt_mstr = concat_mult_ref_tables('punt', yrs_list)
reception_mstr = concat_mult_ref_tables('reception', yrs_list)
rush_mstr = concat_mult_ref_tables('rush', yrs_list)
stadium_mstr = concat_mult_ref_tables('stadium', yrs_list)
team_game_statistics_mstr = concat_mult_ref_tables('team-game-statistics', yrs_list)
team_mstr = concat_mult_ref_tables('team', yrs_list)

        

### Produce a dataset that simply tells us who won each game

In [2]:
# create some new useful datasets
tmp_col_list = ['Team Code', 'Game Code', 'Points']
team_points = team_game_statistics_mstr[tmp_col_list]

# produce a new dataset called game_results_mstr that tells us who won each game
game_results_mstr = game_mstr
game_results_mstr['Visit Points'] = None
game_results_mstr['Home Points'] = None
game_results_mstr['Home Team Winner'] = None
game_results_mstr['Final Spread'] = None
game_results_mstr['Total Points'] = None
for index, row in game_results_mstr.iterrows():
    # find the visiting team's points
    visit_pts_ix = team_points.index[(team_points['Team Code']==row['Visit Team Code']) 
                    & (team_points['Game Code']==row['Game Code'])].tolist()
    visit_pts = team_points.loc[visit_pts_ix[0]].at['Points']
    # find the home team's points
    home_pts_ix = team_points.index[(team_points['Team Code']==row['Home Team Code']) 
                    & (team_points['Game Code']==row['Game Code'])].tolist()
    home_pts = team_points.loc[home_pts_ix[0]].at['Points']
    # calculate the spread and total points
    final_spread = abs(home_pts - visit_pts)
    total_pts = home_pts + visit_pts
    
    # save all values to the game_results dataset
    game_results_mstr.loc[index, 'Visit Points'] = visit_pts
    game_results_mstr.loc[index, 'Home Points'] = home_pts
    game_results_mstr.loc[index, 'Final Spread'] = final_spread
    game_results_mstr.loc[index, 'Total Points'] = total_pts
    if home_pts > visit_pts:
        game_results_mstr.loc[index, 'Home Team Winner'] = 1
    else:
        game_results_mstr.loc[index, 'Home Team Winner'] = 0
    

In [3]:
game_results_mstr.head()

Game Code        Date  Visit Team Code  Home Team Code  \
2005 0   86016420050901  09/01/2005               86             164   
     1  128064820050901  09/01/2005              128             648   
     2  204014020050901  09/01/2005              204             140   
     3  295075420050901  09/01/2005              295             754   
     4  428071920050901  09/01/2005              428             719   

        Stadium Code  Site  Visit Points  Home Points  Home Team Winner  \
2005 0            25  TEAM             0           38                 1   
     1           113  TEAM            15           24                 1   
     2            97  TEAM            26           28                 1   
     3           138  TEAM            26           38                 1   
     4           105  TEAM            41           10                 0   

        Final Spread  Total Points  
2005 0            38            38  
     1             9            39  
     2             2            54  
     3            12            64  
     4            31            51

### Create the master dataset for use in training and validating our game winner model

In [16]:
# Create a dataset that shows each game, by team, and the result 
# essentially double the size of the game_results set by breaking out winners and losers
home_cols = ['Game Code', 'Date', 'Home Team Code', 'Visit Team Code', 'Home Team Winner', 'Site']
away_cols = ['Game Code', 'Date', 'Visit Team Code', 'Home Team Code', 'Home Team Winner', 'Site']
new_cols = ['Game Code', 'Date', 'Team Code', 'Opp Code', 'Won', 'Game_Home']

def flip_winner(i):
    return int(not i)

def site_code(text):
    if text == 'TEAM':
        return 1
    else:
        return 0

# Create two dfs full of team specific results
home_df = game_results_mstr[home_cols]
home_df.columns = new_cols
#home_df['Home'] = 1
home_df['Game_Home'] = home_df['Game_Home'].apply(site_code)

away_df = game_results_mstr[away_cols]
away_df['Home Team Winner'] = away_df['Home Team Winner'].apply(flip_winner)
away_df.columns = new_cols
#away_df['Home'] = 0
away_df['Game_Home'] = 0

# Concatenate the dfs into one final df
gametime_master = pd.concat([home_df, away_df])

# Convert the Date column to Datetime dtype
gametime_master['Date'] = pd.to_datetime(gametime_master['Date'], infer_datetime_format=True)

gametime_master.head()

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

Game Code       Date  Team Code  Opp Code  Won  Game_Home
2005 0   86016420050901 2005-09-01        164        86    1          1
     1  128064820050901 2005-09-01        648       128    1          1
     2  204014020050901 2005-09-01        140       204    1          1
     3  295075420050901 2005-09-01        754       295    1          1
     4  428071920050901 2005-09-01        719       428    0          1

### Write some functions to extract pre-game knowledge and add them to our dataset
#### Team Knowledge (Record, Conf Record, Total Pts, Total Passing Yds, Total Rush Yds, etc)
#### Opponent Knowledge (Same)
#### Concentric Knowledge (previous overlapping opponents)

In [5]:
first_year = '2005'

def get_season_str_yr(gamedate):
    """ Takes the date of a game and 
    returns the season year as a string"""
    if gamedate.month == 1:  # if this is a bowl game
        str_year = str(gamedate.year - 1)
    else:
        str_year = str(gamedate.year)
    return str_year

def season_wins_to_date(team_code, date):
    """Given a team and date, this function returns the number season wins
    up to, but not including, that date.  If this is the first game of the season
    it returns the number of wins from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the full season for this team
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
            wins = last_season['Won'].sum()
        except KeyError:
            wins = 0
        
    else:
        wins = games_to_date['Won'].sum()
    return wins
    
def season_losses_to_date(team_code, date):
    """Given a team and date, this function returns the number season losses
    up to, but not including, that date.  If this is the first game of the season
    it returns the number of losses from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the full season for this team and calculate losses
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
            games = last_season.shape[0]
            wins = last_season['Won'].sum()
            losses = games - wins
        except KeyError:
            losses = 0
    else:        
        wins = games_to_date['Won'].sum()
        losses = games - wins
    return losses
    
def season_record_to_date(team_code, date):
    """Given a team and date, this function returns the season win percentage as a float
    up to, but not including, that date.  If this is the first game of the season
    it returns the percentage from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the full season for this team and calculate wins
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
            games = last_season.shape[0]
            wins = last_season['Won'].sum()
            # don't allow to divide by zero
            if games > 0:
                win_perc = wins / games
            else:
                win_perc = 0
        except KeyError:
            win_perc = 0
    elif ((games == 0) & (str_year == first_year)):
        return 0
    else:
        wins = games_to_date['Won'].sum()
        win_perc = wins / games
    
    return round(win_perc, 3)



In [6]:
def in_conf_game(team_code1, team_code2, yr):
    conf1 = team_mstr[team_mstr['Team Code'] == team_code1].loc[yr].iloc[0]['Conference Code']
    conf2 = team_mstr[team_mstr['Team Code'] == team_code2].loc[yr].iloc[0]['Conference Code']
    if conf1 == conf2:
        return 1
    else:
        return 0
    
def get_norm_stadium_capacity(gamecode):
    """ Given a stadium code and gamedate
    return the normalized value of the stadium capacity"""
    date = pd.Timestamp(game_mstr[game_mstr['Game Code'] == gamecode]['Date'].values[0])
    str_year = get_season_str_yr(date)
    stadium_code = game_mstr[game_mstr['Game Code'] == gamecode]['Stadium Code'].values[0]
    max_stadium = max(stadium_mstr['Capacity'].tolist())
    capacity = stadium_mstr[stadium_mstr['Stadium Code'] == stadium_code].loc[str_year].Capacity.tolist()[0]
    norm_val = int(capacity) / int(max_stadium)
    return norm_val
    

### Add game specific stats to the dataset

In [7]:
gametime_master['Game_Conf'] = gametime_master.apply(lambda x: in_conf_game(x['Team Code'], 
                                                                            x['Opp Code'], 
                                                                            get_season_str_yr(x['Date'])), axis=1)


In [8]:
gametime_master['Game_NZ_Capacity'] = gametime_master.apply(lambda x: 
                                                                      get_norm_stadium_capacity(x['Game Code']), 
                                                                      axis=1)

### Add season win/loss stats for each team to the dataset

In [10]:
gametime_master['WTD'] = gametime_master.apply(lambda x: season_wins_to_date(x['Team Code'], x['Date']), axis=1)
gametime_master['LTD'] = gametime_master.apply(lambda x: season_losses_to_date(x['Team Code'], x['Date']), axis=1)
gametime_master['RTD'] = gametime_master.apply(lambda x: season_record_to_date(x['Team Code'], x['Date']), axis=1)

### Add season win/loss stats for each team's opponent to the dataset

In [11]:
gametime_master['Opp_WTD'] = gametime_master.apply(lambda x: season_wins_to_date(x['Opp Code'], x['Date']), axis=1)
gametime_master['Opp_LTD'] = gametime_master.apply(lambda x: season_losses_to_date(x['Opp Code'], x['Date']), axis=1)
gametime_master['Opp_RTD'] = gametime_master.apply(lambda x: season_record_to_date(x['Opp Code'], x['Date']), axis=1)

In [12]:
gametime_master

Game Code       Date  Team Code  Opp Code  Won  True Home  \
2005 0     86016420050901 2005-09-01        164        86    1          1   
     1    128064820050901 2005-09-01        648       128    1          1   
     2    204014020050901 2005-09-01        140       204    1          1   
     3    295075420050901 2005-09-01        754       295    1          1   
     4    428071920050901 2005-09-01        719       428    0          1   
     5    508049820050901 2005-09-01        498       508    0          1   
     6    529028820050901 2005-09-01        288       529    0          1   
     7    690002820050901 2005-09-01         28       690    1          1   
     8    736074920050901 2005-09-01        749       736    0          1   
     9    771070920050901 2005-09-01        709       771    1          1   
     10   786038820050901 2005-09-01        388       786    1          1   
     11    29073220050902 2005-09-02        732        29    1          1   
     12   306012920050902 2005-09-02        129       306    0          1   
     13     9069420050903 2005-09-03        694         9    1          1   
     14    47031220050903 2005-09-03        312        47    1          1   
     15    51066320050903 2005-09-03        663        51    0          1   
     16    66025720050903 2005-09-03        257        66    1          1   
     17    67007720050903 2005-09-03         77        67    0          1   
     18    71079620050903 2005-09-03        796        71    1          1   
     19    90071620050903 2005-09-03        716        90    1          1   
     20   102010720050903 2005-09-03        107       102    1          1   
     21   110062620050903 2005-09-03        626       110    0          1   
     22   156015720050903 2005-09-03        157       156    1          1   
     23   193019620050903 2005-09-03        196       193    1          1   
     24   207063020050903 2005-09-03        630       207    1          1   
     25   229032820050903 2005-09-03        328       229    1          1   
     26   231032720050903 2005-09-03        327       231    1          1   
     27   255003720050903 2005-09-03         37       255    0          1   
     28   299031120050903 2005-09-03        311       299    1          1   
     29   331041620050903 2005-09-03        416       331    1          1   
...                   ...        ...        ...       ...  ...        ...   
2013 818   66052820131224 2013-12-24         66       528    0          0   
     819  545007120131226 2013-12-26        545        71    1          0   
     820  731050320131226 2013-12-26        731       503    1          0   
     821   77075620131227 2013-12-27         77       756    0          0   
     822  388039220131227 2013-12-27        388       392    1          0   
     823  688042820131227 2013-12-27        688       428    1          0   
     824  140045720131228 2013-12-28        140       457    0          0   
     825  415036720131228 2013-12-28        415       367    0          0   
     826  418032720131228 2013-12-28        418       327    0          0   
     827  587051320131228 2013-12-28        587       513    0          0   
     828   28070020131230 2013-12-30         28       700    0          0   
     829  419072620131230 2013-12-30        419       726    0          0   
     830  433025520131230 2013-12-30        433       255    1          0   
     831  529070320131230 2013-12-30        529       703    1          0   
     832   29006720131231 2013-12-31         29        67    1          0   
     833  193069720131231 2013-12-31        193       697    0          0   
     834  574043020131231 2013-12-31        574       430    0          0   
     835  742011020131231 2013-12-31        742       110    0          0   
     836  128005120140101 2014-01-01        128        51    1          0   
     837  312036520140101 2014-01-01        312       365    0          0   
     838  46

### Write the gametime_master file to a .csv file in the output directory

In [13]:
gametime_master.to_csv('./output/gametime_master.csv')

### Random testing and help code for checking new functions

In [14]:
ck_date = pd.Timestamp(2010, 11, 18, 0)
w = season_wins_to_date(254, ck_date)
l = season_losses_to_date(254, ck_date)
r = season_record_to_date(254, ck_date)
print(w, l, r)

0 0 0


In [15]:
df = gametime_master[gametime_master['Team Code'] == 254]
df.sort_values(by='Date')

Game Code       Date  Team Code  Opp Code  Won  True Home  \
2010 647  254000820101118 2010-11-18        254         8    0          0   
2011 225  254028820110924 2011-09-24        254       288    0          0   
2012 94   254069420120908 2012-09-08        254       694    0          0   
     205  706025420120915 2012-09-15        254       706    0          1   
2013 21   625025420130830 2013-08-30        254       625    0          1   
     136  693025420130907 2013-09-07        254       693    0          1   
     169  254076820130914 2013-09-14        254       768    0          0   
     228  315025420130921 2013-09-21        254       315    0          1   
     333  254000820131005 2013-10-05        254         8    0          0   
     417  716025420131012 2013-10-12        254       716    0          1   
     437  254067020131019 2013-10-19        254       670    0          0   
     492  254049820131026 2013-10-26        254       498    0          0   
     574  772025420131102 2013-11-02        254       772    0          1   
     670  671025420131116 2013-11-16        254       671    0          1   
     703  254003020131123 2013-11-23        254        30    0          0   
     786  646025420131130 2013-11-30        254       646    0          1   

          Home  Game_Conf  Game_Norm_Stadium_Capacity  WTD  LTD  RTD  Opp_WTD  \
2010 647     0          0                    0.947163    0    0  0.0        8   
2011 225     0          0                    0.297672    0    1  0.0        3   
2012 94      0          0                    0.949173    0    1  0.0        1   
     205     1          0                    0.662785    0    1  0.0        1   
2013 21      1          0                    0.662785    0    2  0.0        0   
     136     1          0                    0.662785    0    1  0.0        0   
     169     0          0                    0.590692    0    2  0.0        1   
     228     1          0                    0.662785    0    3  0.0        0   
     333     0          0                    0.947163    0    4  0.0        4   
     417     1          1                    0.662785    0    5  0.0        3   
     437     0          1                    0.279067    0    6  0.0        3   
     492     0          1                    0.283039    0    7  0.0        3   
     574     1          1                    0.662785    0    8  0.0        4   
     670     1          1                    0.662785    0    9  0.0        7   
     703     0          1                    0.288035    0   10  0.0        6   
     786     1          1                    0.662785    0   11  0.0        4   

          Opp_LTD  Opp_RTD  
2010 647        2    0.800  
2011 225        0    1.000  
2012 94         0    1.000  
     205        0    1.000  
2013 21         1    0.000  
     136        1    0.000  
     169        1    0.500  
     228        2    0.000  
     333        0    1.000  
     417        3    0.500  
     437        3    0.500  
     492        4    0.429  
     574        4    0.500  
     670        2    0.778  
     703        4    0.600  
     786        6    0.400